In [ ]:
# Haar Cascade Classifier implementation demonstrates little success in identifying
# generic faces of passers-by. It is suspected that the classifer is trained exclusively
# on frontal face identification instead of profile identification. Also, the training
# data quality is low.
#
# Identified a filter tuned for face profiles.
#
# Instead of implementing a custom VLR or LR Haar cascade or similar, we can relax the
# the problem to first identifying human figures in each of the frames.
#
# The relaxed problem of identifying human figures sees success even without fine
# tuning. The custom LR/VLR face profile can now be implemented as an operation
# on the blocks with detected bodies.
#

import cv2
import sys
import os

targetfolder = 'training_data'
#targetfolder = 'training_data_lower_camera'

videos = os.listdir(targetfolder)
#Get only processed videos
videos = [ targetfolder + '/' + s[:-4]+ '-opencv' for s in videos if os.path.isdir(targetfolder + '/' + s[:-4]+ '-opencv')]

# Get user supplied values
cascPath_facefront = "haarcascade_frontalface_default.xml"
cascPath_faceprofile = "haarcascade_profileface.xml"
cascPath_fullbody = "haarcascade_fullbody.xml"
cascPath_lowerbody = "haarcascade_lowerbody.xml"
cascPath_upperbody = "haarcascade_upperbody.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath_facefront)
profileCascade = cv2.CascadeClassifier(cascPath_faceprofile)
fullbodyCascade = cv2.CascadeClassifier(cascPath_fullbody)
lowerbodyCascade = cv2.CascadeClassifier(cascPath_lowerbody)
upperbodyCascade = cv2.CascadeClassifier(cascPath_upperbody)

#save for manual analysis
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1920,1080))

for imageFolder in videos:
    images = os.listdir(imageFolder)
    for imagePath in images:
        # Read the image
        image = cv2.imread(imageFolder + '/' + imagePath)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Detect faces in the image
        faces = faceCascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3,
                                             minSize=(50, 50))
        profiles = profileCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3,
                                             minSize=(50, 50))
        bodies = fullbodyCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3,
                                             minSize=(100, 200))
        upperbodies = upperbodyCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3,
                                             minSize=(100, 200))
        lowerbodies = lowerbodyCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3,
                                             minSize=(100, 200))

        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        for (x, y, w, h) in profiles:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, 'Face Profile', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        for (x, y, w, h) in bodies:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, 'Body', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        for (x, y, w, h) in upperbodies:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, 'Upper Body', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        for (x, y, w, h) in lowerbodies:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, 'Lower Body', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            
        if len(faces)>0 or len(profiles)>0 or len(bodies)>0 or len(upperbodies)>0 or len(lowerbodies)>0:
            # print("Found {0} nonunique body parts!".format(len(faces)+len(profiles)+len(bodies)+len(upperbodies)+len(lowerbodies)))
            cv2.imshow("Human figure decomposition", image)
            out.write(image)
            # Pause here 0.2 seconds.
            k = cv2.waitKey(1)

            if k == 27:         # If escape was pressed exit
                cv2.destroyWindow("Human figure decomposition")
                break
out.release()
cv2.destroyWindow("Human figure decomposition")